In [6]:
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpDot, lpSum
from utilities import print_result

### Simple Linear Optimization

PuLP can solve the linear optimization problems in the following format.
$$
\min {(3x - 5y)}
$$
$$s.t.
y \leq x 
$$

Here, $3x-5y$ is called the objective function. In this problem, we are trying to minimize this function, while making sure the *constraints* hold. In this problem, there is only one constraint, $y \leq x$.

In [7]:
# problem definition with name and 'sense'
problem = LpProblem('Simple Minimization', LpMinimize)

In [8]:
# Variables are defined with names, bounds, and categories
x = LpVariable('x', lowBound=0, cat='Continuous')
y = LpVariable('y', upBound=5, cat='Integer')

In [9]:
# the objective is added to the problem as a statement
problem += 3 * x - 5 * y

In [10]:
# the constraints are added as conditionals
problem += y <= x

In [11]:
problem.solve()

# problem status is 1 if optimized
print_result(problem)

Optimization status: 1
Final value of the objective: -10.0
Final values of the variables:
x = 5.0
y = 5.0


### Declaring a List of Variables

If there are multiple variables which have the same category, lower and upper bounds, it is possible to use a dictionary and list comprehensions to declare variables, objective, and constraints.

Consider a classic problem of maximixing utility with cost limitations.
$$
\max \sum_{i=1}^4 x_i u_i
$$
$$
\begin{align}
s.t. \sum_{i=1}^4 x_i c_i &\leq 1 \\
0\leq x_i &\leq 20
\end{align}
$$

In [12]:
# A list of variables suffixes (x1, x2, x3, x4)
variable_names = [1, 2, 3, 4]

# Cost of each variables is defined in a dictionary
costs = {
    1: 0.05,
    2: 0.02,
    3: 0.07,
    4: 0.09
}

# Similarly utilities are defined in a dictionary
utilities = {
    1: 10,
    2: 8,
    3: 15,
    4: 20
}

In [14]:
# Variables are defined as a dictionary with comman bounds and category
# variables is a dictionary with keys being the elements of the variable_names list
# Values of this dict is the variable names (x_1, x_2...) which can be referred to define objectives and constraints
variables = LpVariable.dict('x', variable_names, lowBound=0, upBound=20, cat='Continuous')

In [15]:
problem = LpProblem('Maximize Utility', LpMaximize)

In [16]:
# Objective is defined as a dot product of the variables and corresponding utilities
problem += lpDot(variables.values(), utilities.values())

# Similarly, total cost is defined as the dot product of variables and costs
problem += lpDot(variables.values(), costs.values()) <= 1

In [17]:
problem.solve()

print_result(problem)

Optimization status: 1
Final value of the objective: 293.33333400000004
Final values of the variables:
x_1 = 0.0
x_2 = 20.0
x_3 = 0.0
x_4 = 6.6666667


### Integer Programs

Integer Programs: a linear program plus the additional constraints that some or all variables must be integer valued. Variables are called *binary* if they are allowed to take only 0 or 1 values.

### Logical Constraints

Logical constraints with two variables are rather easy.

**Constraint 1:** Either $x_1$ or $x_2$.
$$
x_1 + x_2 \leq 1
$$

Consider the above utility maximization problem. Let's say there are two items with known utilites and costs. Let's denote $x_i$ whether or not the item is bought. Let's try to maximize the utility with non-restricting total cost condition, allowing only one of the items to be selected.
$$
\max \sum_{i=1}^2 x_i u_i
$$
$$
\begin{align}
\sum_{i=1}^2 x_i c_i &\leq 100 \\
x_1 + x_2 &\leq 1
\end{align}
$$

In [12]:
variable_names = [1, 2]

# Costs are low, so even if both item are chosen, the cost limit (<= 100) is satisfied
costs = {
    1: 20,
    2: 10,
}

# Since the utility of 1 item is higher, that is expected to be chosen
utilities = {
    1: 10,
    2: 8
}

problem = LpProblem('Maximize Utility', LpMaximize)
variables = LpVariable.dict('x', variable_names, lowBound=0, upBound=1, cat='Integer')

# Objective
problem += lpDot(variables.values(), utilities.values())

# Cost constraint
problem += lpDot(variables.values(), costs.values()) <= 100

# OR constraint
problem += lpSum(variables.values()) <= 1

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 10.0
Final values of the variables:
x_1 = 1.0
x_2 = 0.0


**Constraint 2:** If $x_1$ then $x_2$.
$$
x_1 \leq x_2
$$
Let's modify the above problem and add a third item. Let's impose the logical condition to be if item 1 is chosen then item 2 has to be chosen.
$$
\max \sum_{i=1}^3 x_i u_i
$$
$$
\begin{align}
\sum_{i=1}^3 x_i c_i &\leq 20 \\
x_1 &\leq x_2
\end{align}
$$

In [22]:
variable_names = [1, 2, 3]

# only one of the item can be chosen due to the cost limitation
costs = {
    1: 15,
    2: 15,
    3: 15,
}

# utility of 1 is highest, but choosing 1 will require choosing 2, but cost limitation restricts that
utilities = {
    1: 20,
    2: 5,
    3: 8,
}

problem = LpProblem('Maximize Utility', LpMaximize)
variables = LpVariable.dict('x', variable_names, lowBound=0, upBound=1, cat='Integer')

# Objective
problem += lpDot(variables.values(), utilities.values())

# Cost constraint
problem += lpDot(variables.values(), costs.values()) <= 20

# IF-THEN constraint
problem += variables.get(1) <= variables.get(2)

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 8.0
Final values of the variables:
x_1 = 0.0
x_2 = 0.0
x_3 = 1.0


Similarly, at least one of $x_1$ and $x_2$ can be imposed with the following condition.
$$
x_1 + x_2 \geq 1
$$
Both, $x_1$ and $x_2$ can be imposed with the following condition.
$$
x_1 + x_2 \geq 2
$$

### Logical Constraints with Non-Binary Variables
Continuous variables are harder to model. Consider the following condition $x \leq 5$ or $x \geq 8$. We introduce a number *M* which is much larger than any variable in the problem and we introduce a binary variable $w$ and rewrite the equations as:
$$
\begin{align}
x &\leq 5 + M(1-w) \\
x &\geq 8 - Mw
\end{align}
$$
Adding a very large number to RHS of a less than or equal to condition automatially satisfies that. Similarly, subtracting a very large number from RHS of a greater than or equal to condition automatically satisfies that. Binary variable $w$ makes sure only one of the equations is satisfied and the other is imposed.

Consider the following problem.
$$
\min x + y
$$
$$
\begin{align}
0 &\leq x \\
0 &\leq y
\end{align}
$$
$$
x \geq 10 \; or \; y \geq 5
$$
We formulate the problem as follows:
$$
\begin{align}
\min x &+ y\\
0 &\leq x \\
0 &\leq y \\
x &\geq 10 - M(1-w)\\
y &\geq 5 - Mw
\end{align}
$$

In [23]:
problem = LpProblem('Minimize', LpMinimize)

x = LpVariable('x', lowBound=0, cat='Continuous')
y = LpVariable('y', lowBound=0, cat='Continuous')
w = LpVariable('w', lowBound=0, upBound=1, cat='Integer')

M = 10

problem += x + y

problem += x >= 10 - M * (1 - w)
problem += y >= 5 - M * w

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 5.0
Final values of the variables:
w = 0.0
x = 0.0
y = 5.0


Since at least one of the conditions has to be satisfied, and condition on y puts a lower limit on the variable and we are trying to minimize the sum, second condition is satisfied and not the first one.